In [1]:
#데이터 삽입
from pymongo import MongoClient

def insert_data():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local  # 'local' 데이터베이스 사용

    # 책 데이터 삽입
    books = [
        {"title": "Kafka on the Shore", "author": "Haruki Murakami", "year": 2002},
        {"title": "Norwegian Wood", "author": "Haruki Murakami", "year": 1987},
        {"title": "1Q84", "author": "Haruki Murakami", "year": 2009}
    ]
    db.books.insert_many(books)

    # 영화 데이터 삽입
    movies = [
        {"title": "Inception", "director": "Christopher Nolan", "year": 2010, "rating": 8.8},
        {"title": "Interstellar", "director": "Christopher Nolan", "year": 2014, "rating": 8.6},
        {"title": "The Dark Knight", "director": "Christopher Nolan", "year": 2008, "rating": 9.0}
    ]
    db.movies.insert_many(movies)

    # 사용자 행동 데이터 삽입
    user_actions = [
        {"user_id": 1, "action": "click", "timestamp": "2023-04-12T08:00:00Z"},
        {"user_id": 1, "action": "view", "timestamp": "2023-04-12T09:00:00Z"},
        {"user_id": 2, "action": "purchase", "timestamp": "2023-04-12T10:00:00Z"}
    ]
    db.user_actions.insert_many(user_actions)

    print("Data inserted successfully")
    client.close()

if __name__ == "__main__":
    insert_data()


Data inserted successfully


In [2]:
client = MongoClient('mongodb://localhost:27017/')
db = client.local

In [14]:
# [문제 1: 특정 장르의 책 찾기]
# 데이터베이스에 새로운 필드로 genre를 책 데이터에 추가
# "fantasy" 장르에 해당하는 모든 책의 제목과 저자를 찾는 MongoDB 쿼리를 함수로 만들어 문제를 해결
def book_genre():
    collection=db['books']
    query={"genre":"fantasy"}
    projection={"title":1,"author":1}
    for doc in collection.find(query,projection):
        print(doc)

book_genre()    #genre의 추가 데이터X

In [23]:
# [문제 2: 감독별 평균 영화 평점 계산]
# 모든 영화 감독의 영화 평점 평균을 계산하고, 평균 평점이 높은 순으로 정렬하는 MongoDB 쿼리를 함수로 만들어 문제를 해결
def director_avg():
    collection=db['movies']
    for doc in collection.aggregate([{"$group":{"_id":"$director","rating_avg":{"$avg":"$rating"}}}]):
        print(doc)

director_avg()

{'_id': 'Christopher Nolan', 'rating_avg': 8.799999999999999}


In [19]:
# [문제 3: 특정 사용자의 최근 행동 조회]
# 사용자 ID가 1인 사용자의 최근 행동 5개를 시간 순으로 정렬하여 조회하는 MongoDB 쿼리를 함수로 만들어 문제를 해결
def action_find():
    collection=db['user_actions']
    query={"user_id":1}
    for doc in collection.find(query).sort("timestamp",1).limit(5):
        print(doc)

action_find()

{'_id': ObjectId('6833f9e8c6d3d37b106f4d59'), 'user_id': 1, 'action': 'click', 'timestamp': '2023-04-12T08:00:00Z'}
{'_id': ObjectId('6833f9e8c6d3d37b106f4d5a'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}


In [24]:
# [문제 4: 출판 연도별 책의 수 계산]
# 각 출판 연도별로 출판된 책의 수를 계산하고, 출판된 책의 수가 많은 순서대로 정렬하는 MongoDB 쿼리를 함수로 만들어 문제를 해결

def book_year():
    collection=db['books']
    group={"$group":{"_id":"$year","total":{"$sum":1}}}
    for doc in collection.aggregate([group,{"$sort":{"total":-1}}]):
        print(doc)

book_year()

{'_id': 2002, 'total': 1}
{'_id': 1987, 'total': 1}
{'_id': 2009, 'total': 1}


In [25]:
# [문제 5: 특정 사용자의 행동 유형 업데이트]
# 사용자 ID가 1인 사용자의 2023년 4월 10일 이전의 "view" 행동을 "seen"으로 변경하는 MongoDB 업데이트 쿼리를 함수로 만들어 문제를 해결
def action_update():
    collection=db['user_actions']
    query={"user_id":1,"timestamp":{"$lt":"2023-04-10"}}
    update={"$set":{"action":"seen"}}
    collection.update_many(query,update)

action_update() #4월 10일 이전의 기록이 없음
collection=db['user_actions']
for doc in collection.find():
    print(doc)

{'_id': ObjectId('6833f9e8c6d3d37b106f4d59'), 'user_id': 1, 'action': 'click', 'timestamp': '2023-04-12T08:00:00Z'}
{'_id': ObjectId('6833f9e8c6d3d37b106f4d5a'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}
{'_id': ObjectId('6833f9e8c6d3d37b106f4d5b'), 'user_id': 2, 'action': 'purchase', 'timestamp': '2023-04-12T10:00:00Z'}
